In [4]:
"""Backend supported: tensorflow.compat.v1, tensorflow, pytorch, paddle"""
import deepxde as dde
import matplotlib.pyplot as plt
import numpy as np
import sys

AttributeError: module 'numpy.random' has no attribute 'mtrand'

In [ ]:
# we add path for your data
sys.path.append('/home/janis/SCIML/summerschool')

In [ ]:
kappa = 0.01
c = 0.5

In [ ]:
def equation(x, y, f):
    dy_xx = dde.grad.hessian(y, x)
    dy_dx = dde.grad.jacobian(y, x)
    return -kappa*dy_xx + c*dy_dx - f

In [ ]:
# Domain is interval [0, 1]
geom = dde.geometry.Interval(0, 1)

In [ ]:
def u_boundary(_):
    return 0

In [ ]:
def boundary(_, on_boundary):
    return on_boundary

In [ ]:
bc = dde.icbc.DirichletBC(geom, u_boundary, boundary)
pde = dde.data.PDE(geom, equation, bc, num_domain=100, num_boundary=2)

In [ ]:
# AJOUT DE VOTRE DATASET
folder_path = "/home/janis/SCIML/summerschool/data/benchmarks/given/"
x_coords = np.load(folder_path + 'x.npy')
f_train_data = np.load(folder_path + 'f_train_data.npy')
u_train_data = np.load(folder_path + 'u_train_data_c0.5.npy')

In [ ]:
print(f"Data shapes: x {x_coords.shape}, f {f_train_data.shape}, u {u_train_data.shape}")

In [ ]:
# we create custom function space with your data
class YourDataSpace:
    def __init__(self, functions_data):
        self.functions = functions_data
        self.n_functions = len(functions_data)
    
    def random(self, n):
        indices = np.random.choice(self.n_functions, size=n, replace=False)
        return self.functions[indices]
    
    def eval_batch(self, features, points):
        return features

In [ ]:
space = YourDataSpace(f_train_data[:1000])  # we use subset
evaluation_points = x_coords.reshape(-1, 1)
num_eval_points = len(evaluation_points)

In [ ]:
# Define PDE operator with your data
pde_op = dde.data.PDEOperatorCartesianProd(
    pde,
    space,
    evaluation_points,
    num_function=200,
)

In [ ]:
# Setup DeepONet
dim_x = 1
p = 32
net = dde.nn.DeepONetCartesianProd(
    [num_eval_points, 32, p],
    [dim_x, 32, p],
    activation="tanh",
    kernel_initializer="Glorot normal",
)

In [ ]:
# Define and train model
model = dde.Model(pde_op, net)
dde.optimizers.set_LBFGS_options(maxiter=1000)
model.compile("L-BFGS")
model.train()

In [ ]:
# Test with your actual data
n_test = 40
test_indices = np.random.choice(len(f_train_data), size=n_test, replace=False)
test_f = f_train_data[test_indices]
test_u_true = u_train_data[test_indices]

In [ ]:
# we predict
x_test = evaluation_points
predictions = []
for i in range(n_test):
    pred = model.predict((test_f[i:i+1], x_test))
    predictions.append(pred.flatten())

In [ ]:
# we plot comparison with your real data
fig, axes = plt.subplots(2, n_test, figsize=(15, 8))

In [ ]:
for i in range(n_test):
    # Plot your source functions f(x)
    axes[0, i].plot(x_coords, test_f[i], 'b-', linewidth=2)
    axes[0, i].set_title(f'Your f(x) - Test {i+1}')
    axes[0, i].grid(True)
    
    # Plot comparison: your true solutions vs predictions
    axes[1, i].plot(x_coords, test_u_true[i], 'r-', label='Your true u(x)', linewidth=2)
    axes[1, i].plot(x_coords, predictions[i], 'g--', label='DeepONet pred', linewidth=2)
    axes[1, i].set_title(f'Solutions u(x) - Test {i+1}')
    axes[1, i].legend()
    axes[1, i].grid(True)

In [ ]:
plt.tight_layout()
plt.show()

In [ ]:
# we compute errors with your data
for i in range(n_test):
    mse = np.mean((test_u_true[i] - predictions[i])**2)
    print(f"Test {i+1} MSE vs your data: {mse:.6f}")

In [ ]:
print("Training with your data completed!")